## Calculating Strategy Performance

In this notebook I want to define one function calculating the performace of a strategy and put them in a file called `Utils.py` for future usage.

Once decided my strategy, I use the function `backtest_portfolio` to have the porfolio value over time as well as the asset values over time. From here I calulate the strategy performance.

In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
plt.style.use('ggplot')

In [3]:
# importing necessary functions from Utils
from Utils import prices, portfolio_prices, backtest_portfolio

#### 1. Total Return

$$\text{Total Return} = \frac{P_{end} - P_{start}}{P_{start}}$$

$P_{start}$ is my initial investment or the porftolio value at $t=0$; $P_{end}$ is the portfolio value at the end.
Assuming no dividend distribution, $P_{end} - P_{start}$ is the total gain or loss from my investment.

I define my strategy: assets to include in porfolio and their initial weights. Plus I decide to follow a *buy-and-hold* strategy.

In [4]:
# asset's tickers
tickers = ['CSSPX','EM710','ITPS','PHAU']
# inital weights assigned per ticker
initial_weights = [1/len(tickers) for tick in tickers]
print(initial_weights)

[0.25, 0.25, 0.25, 0.25]


In [5]:
# taking single asset prices
data = []
for tick in tickers:
    data.append(prices(ticker=tick))

# structuring asset prices in a DataFrame
pf_prices = portfolio_prices(data)

pf_prices.head(3)

,CSSPX,EM710,ITPS,PHAU
Date,,,,
2010-05-19,72.849998,122.160004,133.350006,95.830002
2010-05-20,72.849998,122.199997,132.089996,95.070000
2010-05-21,72.849998,122.349998,130.199997,93.260002


Now I backtest my strategy, that is I calculate my porftolio value over time based on the historical asset prices in the `pf_prices` DataFrame.

In [6]:
pf_val = backtest_portfolio(prices=pf_prices, weights=initial_weights, rebalance=False)
pf_val.tail(3)

,CSSPX,EM710,ITPS,PHAU,Portfolio
Date,,,,,
2025-04-09,1.646294,0.339677,0.412561,0.678545,3.077078
2025-04-10,1.712354,0.340864,0.411024,0.688459,3.152701
2025-04-11,1.691215,0.341274,0.397919,0.693572,3.123979


In [7]:
tot_return = pf_val.Portfolio.iloc[-1]/pf_val.Portfolio.iloc[0] - 1 
print('Portfolio Total Return (%):', round(tot_return*100,2))

Portfolio Total Return (%): 212.4


In [8]:
tot_return_assets = pf_val[tickers].iloc[-1]/pf_val[tickers].iloc[0] - 1
print('Asset Total Returns (%):')
print((tot_return_assets*100).round(2))

Asset Total Returns (%):
CSSPX    576.49
EM710     36.51
ITPS      59.17
PHAU     177.43
dtype: float64


You can notice that the Portfolio Total Return is the weighted average of asset Total Returns, weighted by assets `initial_weights`:

In [9]:
print('Portfolio Total Return (%):', round((tot_return_assets.dot(initial_weights))*100,2))

Portfolio Total Return (%): 212.4


Let's see how the performance changes, if I introduce rebalancing every 252 trading days in my strategy.

In [10]:
pf_val = backtest_portfolio(prices=pf_prices, weights=initial_weights, rebalance=True, rebalance_freq=252)
tot_return = pf_val.Portfolio.iloc[-1]/pf_val.Portfolio.iloc[0] - 1 
print('Portfolio Total Return (%) with rebalance:', round(tot_return*100,2))

Portfolio Total Return (%) with rebalance: 170.96


#### 2. Annual Return

$$\text{Annual Return} = (1 + \text{Total Return})^{\frac{1}{\text{Years}}} - 1$$

$\text{Years}$ is the nr of years of the investement from the start.

In [11]:
pf_years = (pf_val.index[-1] - pf_val.index[0])/pd.to_timedelta('365.25D') # length in years
annual_return = ((1 + tot_return)**(1/pf_years))-1 # calculate annual return
print('Portfolio Annual Return (%) with rebalance:', round(annual_return*100,2))

Portfolio Annual Return (%) with rebalance: 6.92


In [12]:
# calculating annual return for buy-and-hold strategy
pf_val = backtest_portfolio(prices=pf_prices, weights=initial_weights, rebalance=False)
tot_return = pf_val.Portfolio.iloc[-1]/pf_val.Portfolio.iloc[0] - 1
pf_years = (pf_val.index[-1] - pf_val.index[0])/pd.to_timedelta('365.25D') # length in years
annual_return = ((1 + tot_return)**(1/pf_years))-1 # calculate annual return
print('Portfolio Annual Return (%) buy-and-hold:', round(annual_return*100,2))

Portfolio Annual Return (%) buy-and-hold: 7.95


We can see that the two strategies have also different Annual Returns, even if portfolio assets and initial weights are the same.

#### 3. Annual Volatility

Annual Volatity is the standard deviation of annual returns. It's calculated from the standard deviation of the daily returns multiplied by the square root of the 252 trading days in a year.

In [13]:
annual_volatility = np.std((pf_val.Portfolio).pct_change().dropna())*np.sqrt(252) 
print('Portfolio Annual Volatility (%) buy-and-hold:', round(annual_volatility*100,2))

Portfolio Annual Volatility (%) buy-and-hold: 8.68


In [14]:
# calculating annual return for rebalancing strategy
pf_val = backtest_portfolio(prices=pf_prices, weights=initial_weights, rebalance=True, rebalance_freq=252)
annual_volatility = np.std((pf_val.Portfolio).pct_change().dropna())*np.sqrt(252) 
print('Portfolio Annual Volatility (%) with rebalance:', round(annual_volatility*100,2))

Portfolio Annual Volatility (%) with rebalance: 7.18


*Buy-and-hold* strategy seems to have higher annual volatity and retruns than the *rebalance* strategy.

#### 4. Sharpe Ratio

Which of the two strategy to choose? 
- *Buy-and-hold* strategy offered higher annual returns in the past at the price of higher risk (volatility).
- *Rebalance* strategy offered lower annual returns in the past but with lower risk.

Sharpe Ratio measures how much annual return we get per each 1 point of annual volatility:
$$\text{Sharpe Ratio} = \frac{\text{Annual Return}}{\text{Annual Volatility}}$$



In [15]:
# Backtesting both strategies
pf_val_bh = backtest_portfolio(prices=pf_prices, weights=initial_weights, rebalance=False) # buy and hold
pf_val_reb = backtest_portfolio(prices=pf_prices, weights=initial_weights, 
                                rebalance=True, rebalance_freq=252) # rebalance

# calculating total returns of both strategies
tot_return_bh = pf_val_bh.Portfolio.iloc[-1]/pf_val_bh.Portfolio.iloc[0] - 1 # buy-and-hold
tot_return_reb = pf_val_reb.Portfolio.iloc[-1]/pf_val_reb.Portfolio.iloc[0] - 1 # rebalance

# calculating annual returns of both strategies
pf_bh_years = (pf_val_bh.index[-1] - pf_val_bh.index[0])/pd.to_timedelta('365.25D') # length in years buy-and-hold
pf_reb_years = (pf_val_reb.index[-1] - pf_val_reb.index[0])/pd.to_timedelta('365.25D') # length in years rebalance
annual_return_bh = ((1 + tot_return_bh)**(1/pf_bh_years))-1 # calculate annual return buy-and-hold
annual_return_reb = ((1 + tot_return_reb)**(1/pf_reb_years))-1 # calculate annual return rebalance

# calculating annual volatilities of both strategies
annual_volatility_bh = np.std((pf_val_bh.Portfolio).pct_change().dropna())*np.sqrt(252) # buy-and-hold
annual_volatility_reb = np.std((pf_val_reb.Portfolio).pct_change().dropna())*np.sqrt(252) # rebalance

In [16]:
# calculating Sharpe Ratios of both strategies:
print('Buy-and-hold Sharpe Ratio:', round(annual_return_bh/annual_volatility_bh, 2))
print('Lenght of the buy-and-hold backtest:', round(pf_bh_years,1), 'years')
print('Rebalance Sharpe Ratio:', round(annual_return_reb/annual_volatility_reb, 2))
print('Lenght of the rebalance backtest:', round(pf_reb_years,1), 'years')

Buy-and-hold Sharpe Ratio: 0.92
Lenght of the buy-and-hold backtest: 14.9 years
Rebalance Sharpe Ratio: 0.96
Lenght of the rebalance backtest: 14.9 years


The higher than 1.0 the Sharpe Ratio the better the stratey is, meaning that we are more highly rewarded per each point of risk (volatility) taken. If Sharpe Ratio < 1, we are taking more risks than what we get back as returns.

Personally, I prefer strategies with the highest Sharpe Ratio (possibly > 1.0).

#### 5. Putting all together and write a function calculating performance indicators

I'm introducing also the optional parameter of `risk_free_rate` to calculate the Sharpe Ratio. In fact, Sharpe Ratio numerator is the excess return of the investment over a risk free investment. 

In the example in paragraph 4. I didn't take into consideration the `risk_free_rate`, because there are almost no risk free investements and therefore not making calculation more complicated.

In [17]:
def portfolio_performance(pf: pd.Series, risk_free_rate: float = 0.0) -> pd.Series:
    """
    Calculates portfolio performance indicators from a time series of daily portfolio values.

    Parameters:
    - pf: pd.Series with portfolio values indexed by date
    - risk_free_rate: Annual risk-free rate (default = 0.0)

    Returns:
    - pd.Series with Total Return, Annual Return, Annual Volatility, and Sharpe Ratio
    """
    indicators = ['Total_Return', 'Annual_Return', 'Annual_Volatility', 'Sharpe']
    perf = pd.Series(index=indicators, dtype='float64')

    # Length of backtest in years
    pf_years = (pf.index[-1] - pf.index[0]) / pd.to_timedelta('365.25D')

    # Daily returns
    daily_returns = pf.pct_change().dropna()

    # Metrics
    total_return = pf.iloc[-1] / pf.iloc[0] - 1
    annual_return = (1 + total_return) ** (1 / pf_years) - 1
    annual_volatility = daily_returns.std() * np.sqrt(252)

    # Sharpe Ratio (excess return over volatility)
    excess_return = annual_return - risk_free_rate
    sharpe_ratio = excess_return / annual_volatility if annual_volatility != 0 else np.nan

    # Fill output
    perf['Total_Return'] = total_return
    perf['Annual_Return'] = annual_return
    perf['Annual_Volatility'] = annual_volatility
    perf['Sharpe'] = sharpe_ratio

    return perf

In [18]:
# from previous backtests for rebalance
metrics = portfolio_performance(pf_val_reb.Portfolio)
print(metrics)


Total_Return         1.709636
Annual_Return        0.069205
Annual_Volatility    0.071779
Sharpe               0.964137
dtype: float64


In [19]:
# from previous backtests for buy-and-hold
metrics = portfolio_performance(pf_val_bh.Portfolio)
print(metrics)

Total_Return         2.123979
Annual_Return        0.079467
Annual_Volatility    0.086843
Sharpe               0.915065
dtype: float64


Each single indicator can be individually accessed from the `metrics` pd.Series:

In [20]:
metrics.Total_Return, metrics.Annual_Return, metrics.Annual_Volatility, metrics.Sharpe

(2.123979494199022,
 0.07946698968321697,
 0.08684297711092118,
 0.915065239895183)

### Summary

In this chapter we have seen four common portfolio performance indicators and defined the function `portfolio_performance()` calculating these indicators by passing it a `pd.Series` containing backtest portfolio values.

Now I want to cover another aspect related to risk: *maximum drawdown* when testing a portfolio strategy.